# Run Functions to Add Information to Projects

In [1]:
# ! pip install nltk

In [1]:

import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

import _data_utils
import _script_utils

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [3]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [4]:
### replace the file path with the new data file
### Latest is from 6/29/23
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_062923_ToDLA.xlsx",))

### last data export 3/9/2023
# proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/IIJA_FMIS_AllProject_20230309_ToDLA.xlsx"))
                       


In [5]:

# df = utils.read_data_all()
proj = _data_utils.add_new_codes(proj)
    
## function that adds known agency name to df 
df = _script_utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [6]:
df.sample(3)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
891,2023-02-22,Y003,Projects to Reduce PM 2.5 Emissions,5216058,0621000082L,"NINE VARIOUS ALLEYS BETWEEN NORTH, G, EAST, DUFF, COLUMBIA, PONDEROSA, AND CYPRESS. PAVE DIRT ALLEY WAYS",19,Cong Dist 22,Active,"NINE VARIOUS ALLEYS BETWEEN NORTH, G, EAST, DUFF, COLUMBIA, PONDEROSA, AND CYPRESS. PAVE DIRT ALLEY WAYS",44,Other,260544.00,95427.00,L5216FCOG,5216.0,Reedley,6.0,Fresno County,Council of Fresno County Governments,Council Of Fresno County Goverments
2522,2023-05-31,ER01,Emergency Relieve Funding,29S9001,0121000086S,IN DEL NORTE COUNTY ABOUT 10 MI SOUTH OF CRESCENT CITY FROM 1.3 MI TO 1.9 MI NORTH OF RUDISILL ROAD LANDSLIDE REPAIR,45,Cong Dist 2,Active,ON STATE ROUTE: 101. IN DEL NORTE COUNTY ABOUT 10 MI SOUTH OF CRESCENT CITY FROM 1.3 MI TO 1.9 MI NORTH OF RUDISILL ROAD LANDSLIDE REPAIR,6,4R - Restoration & Rehabilitation,9299267.00,8232641.00,S ER NONE,NaN,California,1.0,Mendocino County,NaN,NaN
557,2022-06-21,Y001,National Highway Performance Program (NHPP),P060162,0716000063S,"ROUTE 060 PM 28.0/30.4: IN LOS ANGELES COUNTY, IN AND NEAR POMONA, FROM PHILLIPS RANCH ROAD TO RESERVOIR STREET. CONSTRUCT AND INSTALL STORMWATER TRE",37,Cong Dist 35,Active,"ON STATE ROUTE: 60. ROUTE 060 PM 28.0/30.4: IN LOS ANGELES COUNTY, IN AND NEAR POMONA, FROM PHILLIPS RANCH ROAD TO RESERVOIR STREET. CONSTRUCT AND INSTALL STORMWATER TREATMENT BMPS INCLUDING BIOFILTRATION SWALES/STRIPS, DETENTION BASIN AND MEDIA FILTERS. CONSTRUCT AND INSTALL STORMWATER TREATMENT BMPS INCLUDING BIOFILTRATION SWALES/STRIPS, DETENTION BASIN AND MEDIA FILTERS.",42,Training,1658.91,1463.74,S SCAG,NaN,California,7.0,Los Angeles County,NaN,NaN


## Test & Export

In [7]:
### run the data through the rest of the script
### return a dataset that is aggregated at the project and program code

agg = _script_utils.get_clean_data(df, full_or_agg = 'agg')

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/home/jovyan/data-analyses/dla/iija/_script_utils.py:452: FutureWarning: The default value of regex will change from True to False in a future version.
  df['implementing_agency_locode'] = df['implementing_agency_locode'].str.replace('.0', '')


In [8]:
agg.sample(5)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,county_name_abbrev,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
829,2023-02-27,5006839,Los Angeles,L5006SCAG,Y110,Bridge Formula Program,0716000279L,11,Bridge Replacement - No Added Capacity,"SIXTH STREET VIADUCT OVER LA RIVER, US 101, AND UPRR, BNSF, AMTRACK, LACMTA AND METROLINK TRACKS BRIDGE # 53C-1880 AND STATE BRIDGE # 53-0595THIS PRO",38220897,|34|,|07|,37,Los Angeles County,|LA|,06,Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments,Bridge Replacement in Los Angeles,"Bridge Replacement in Los Angeles, part of the Bridge Formula Program. (Federal Project ID: 5006839)."
1310,2023-06-14,0057100,California,S NON-MPO,Y001,National Highway Performance Program (NHPP),0314000161S,17,Construction Engineering,IN GLENN COUNTY 4.8 MILES NORTH OF WILLOWS AT THE WILLOWS SAFETY ROADSIDE REST AREA UPGRADE WATER DISTRIBUTION AND WASTEWATER SYSTEM.,663882,|03|,|03|,21,Glenn County,|GLE|,nan,None,None,Construction Engineering Projects in Glenn County,"Construction Engineering Projects in Glenn County, part of the National Highway Performance Program (NHPP). (Federal Project ID: 0057100)."
10,2022-01-24,39R0001,California,S ER NONE,ER01,Emergency Relieve Funding,0820000004S,15|16|17,Preliminary Engineering|Right of Way|Construction Engineering,IN SAN BERNARDINO COUNTY ON STATE ROUTE 178 FROM 2.35 MILES WEST OF TRONA ROAD TO 1.068 MILES EAST OF SALT WELLS CANYON REPAIR ROADWAY DAMAGE AND SLO,155129,|08|,|08|,71,San Bernardino County,|SBD|,nan,None,None,Preliminary Engineering Projects in San Bernardino County,"Preliminary Engineering Projects in San Bernardino County, part of the Emergency Relieve Funding. (Federal Project ID: 39R0001)."
630,2022-12-01,X073142,California,S SANDAG,Y240,Surface Transportation Block Grant,1114000056S,17,Construction Engineering,IN SAN DIEGO COUNTY AT VARIOUS LOCATIONS CONSTRUCT AMERICANS WITH DISABILITIES ACT RAMPS,154035,|01|,|nan|,999,Statewide,|NA|,nan,None,None,Construct Ramp in San Diego County,"Construct Ramp in San Diego County, part of the Surface Transportation Block Grant. (Federal Project ID: X073142)."
191,2022-05-18,P138058,California,S SCAG,Y001,National Highway Performance Program (NHPP),0713000217S,16,Right of Way,"IN LOS ANGELES COUNTY, AT LITTLEROCK, FROM 87TH STREET EAST TO 0.2 MILE EAST OF 96TH STREET EAST WIDEN CONVENTIONAL HIGHWAY (SEG 6) (TC)",200000,|25|,|07|,37,Los Angeles County,|LA|,nan,None,None,Widen Road in Los Angeles County,"Widen Road in Los Angeles County, part of the National Highway Performance Program (NHPP). (Federal Project ID: P138058)."


In [9]:
## when grouped by funding program, len is 1612
len(agg)

1612

In [14]:
agg>>count(_.project_number)>>filter(_.n>1)>>arrange(-_.n)

,project_number,n
506,5004049,4
1115,6054094,4
38,0010203,3
41,0010206,3
44,0010209,3
...,...,...
1427,X023043,2
1444,X059084,2
1454,X075048,2
1460,X093010,2


In [15]:
agg>>filter(_.project_number=='5004049')

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,county_name_abbrev,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
1493,2023-06-26,5004049,San Diego,L5004SANDAG,Y001,National Highway Performance Program (NHPP),11955780L,10,Bridge Replacement - Added Capacity,"WEST MISSION BAY DRIVE OVER THE SAN DIEGO RIVER BRIDGE REPLACEMENT, BR. NO. 57C-0023",38128417,|52|,|11|,73,San Diego County,|SD|,04,San Diego Association of Governments,San Diego Association Of Governments,Replace Bridge in San Diego,"Replace Bridge in San Diego, part of the National Highway Performance Program (NHPP). (Federal Project ID: 5004049)."
1494,2023-06-26,5004049,San Diego,L5004SANDAG,Y110,Bridge Formula Program,11955780L,10,Bridge Replacement - Added Capacity,"WEST MISSION BAY DRIVE OVER THE SAN DIEGO RIVER BRIDGE REPLACEMENT, BR. NO. 57C-0023",11871582,|52|,|11|,73,San Diego County,|SD|,04,San Diego Association of Governments,San Diego Association Of Governments,Replace Bridge in San Diego,"Replace Bridge in San Diego, part of the Bridge Formula Program. (Federal Project ID: 5004049)."
1495,2023-06-26,5004049,San Diego,L5004SANDAG,Y908,Bridge Replacement and Rehabilitation Program,11955780L,10|17,Bridge Replacement - Added Capacity|Construction Engineering,"WEST MISSION BAY DRIVE OVER THE SAN DIEGO RIVER BRIDGE REPLACEMENT, BR. NO. 57C-0023",9711359,|52|,|11|,73,San Diego County,|SD|,04,San Diego Association of Governments,San Diego Association Of Governments,Replace Bridge in San Diego,"Replace Bridge in San Diego, part of the Bridge Replacement and Rehabilitation Program. (Federal Project ID: 5004049)."
1496,2023-06-26,5004049,San Diego,L5004SANDAG,Y909,nan,11955780L,10,Bridge Replacement - Added Capacity,"WEST MISSION BAY DRIVE OVER THE SAN DIEGO RIVER BRIDGE REPLACEMENT, BR. NO. 57C-0023",10004189,|52|,|11|,73,San Diego County,|SD|,04,San Diego Association of Governments,San Diego Association Of Governments,Replace Bridge in San Diego,"Replace Bridge in San Diego, part of the nan. (Federal Project ID: 5004049)."


In [12]:
### pretty print the column names

agg = _script_utils.title_column_names(agg)

In [14]:
#### export new file to the GCS bucket
# agg.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_082323_rerun.csv")

### old export
# agg.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_agg.csv")


In [13]:
# ###test full. PASS title, no concat and 1241 rows
# full = _script_utils.get_clean_data(df, full_or_agg = 'full')